In [1]:
# Imports
from rocketpy import Environment, HybridMotor, Rocket, Flight
import datetime

In [1]:
# TODO : Automate data with csv on drive
# TODO : Get missing data values

In [ ]:
# Chargement données pour le moteur hybride
SOURCE    = "HYB30kpar.eng"
NAME      = "Kraken2.0"
BURN_TIME = 0 # ?
COORDS_ORIENTATION ="nozzle_to_combustion_chamber"

# Masse solide
DRY_MASS    = 16940/1000
DRY_INERTIA = 0 # Inertie de la masse solide (liste en kg*m^2)
CM_DRY      = 0 # Centre de masse de la masse solide (m)

NOZZLE_RADIUS = 0.0760476
THROAT_RADIUS = 0 # ? Rayon du throat du nozzle. Pas nécessaire dans certains simulations

# Grain data
GRAIN_N         = 1           # Nombre de grains
GRAIN_DENSITY   = 0.503381099 # Densité du grain (kg/m^3)
GRAIN_ORAD      = 0.063881    # rayon extérieur du grain (m)
GRAIN_INIT_IRAD = 0.030988    # rayon initial intérieur (m)
GRAIN_INIT_H    = 0.22225     # Hauteur initiale (m)
GRAIN_SEP       = 0           # ? Distance entre 2 grains
GRAIN_CM        = 0           # ? Centre de masse (m)

In [ ]:
# Chargement données pour l'environnement du vol
DATE      = datetime.datetime(2023, 11, 21) # Date du launch (Y, M, D)
ELEVATION = 4595    # Élévation du site de launch
GEO_DATUM = 'NAD83' # Référence spatiale pour le Canada (correction de données)
TIMEZONE  = 'GMT-5'
MAX_H     = 30000   # Hauteur maximal attendu en m. Utilisé pour  ne pas avoir un surplus de données
LAT       = 0       # Latitude du spaceport
LON       = 0       # Longitude du spaceport

In [ ]:
# Chargement données de la fusée
# Construction fusée
ROCKET_RAD     = 0.18           # le rayon maximal de la fusée (excluant les ailettes) en mètres
ROCKET_MASS    = 0              # Masse de la fusée sans le moteur (kg)
ROCKET_INERTIA = 0              # liste des inerties. Axes perpendiculaires : I11, I22. Axe central : I33
ROCKET_CM      = 0              # Centre de masse de la fusée sans le moteur
ROCKET_COORDS  = "nose_to_tail" # Référence pour tous les données de type mesures (longeur, cm, etc.)

# Nose cone
NC_L     = 0.79          # Longueur du nose cone
NC_KIND  = "von karman"  # Forme du nose cone
NC_POS   = 0             # Position du nose cone (0 car nose_to_tail)
NC_BLUFF = 0             # Ratio entre le rayon du tip de l'ogive et le rayon à la base de l'ogive. Optionnel

# Ailettes
FIN_N     = 3        # nombre d'ailettes
FIN_SPAN  = 0.17     # Hauteur des ailettes en mètres
FIN_RC    = 0.437134 # Root chord des ailettes en mètres
FIN_TC    = 0.127    # Tip chord des ailettes en mètres
FIN_POS   = 0        # Position du top des ailettes à partir du nose cone en mètres
FIN_SWEEP = 0.249174 # Sweep length en mètres

# Boattail
TAIL_TOP    = 0.174244 # Rayon du top du tail en mètres
TAIL_BOTTOM = 0.092456 # Rayon du bottom du tail en mètres
TAIL_L      = 0.058674 # Longueur du tail en mètres
TAIL_POS    = 0        # Position du tail selon le nose cone en mètres

# Parachutes
MAIN_NAME  = "Main Parachute" # Nom du parachute principal
MAIN_CD    = 0 # F = ((1/2)*rho*V^2)*cd_s === Coefficient drag * aire référence. Utilisé pour calculer la force du drag
MAIN_EJECT = 0 # Hauteur d'éjection du parachute en mètres
MAIN_LAG   = 0 # Temps entre trigger et éjection en secondes (optionel)

DRAG_NAME  = "Drag Parachute"
DRAG_CD    = 0
DRAG_EJECT = 0
DRAG_LAG   = 0

# Rails
BUTTON_TOP    = 0 # Position du bouton le plus éloigné du nozzle (selon nose cone) (m)
BUTTON_BOTTOM = 0 # Position du bouton le plus proche du nozzle (selon nose cone) (m)
BUTTON_ANG    = 0 # Position angulaire des boutons en degrés (selon axes symmétriques de la fusée relative aux axes principales)

In [ ]:
# Chargement données pour la simulation du vol
RAIL_L = 0 # Longueur de la surface entre la fusée et le rail
RAIL_INC = 0 # Inclinaison du rail relative au sol en degrés
RAIL_HEAD = 0 # Angle relative au nord en degrés